# 지역 특성을 반영한 계약 건수 및 업무량 예측

- **지역 (`city`)**: 분석 대상이 되는 도시 이름  
- **연도 (`year`)**: 데이터의 기준 연도  
- **지역 인구수 (`regional_population`)**: 각 도시의 인구수  
- **신규 유입 인구수 (`new_inflow_population`)**: 연도별 신규 유입 인구수  
- **저소득 고령자 인구수 (`low_income_elderly_population`)**: 저소득 고령자의 수  
- **한부모 가정 수 (`single_parent_households`)**: 한부모 가정의 수  
- **생계의료비 수급자 수 (`low_income_assistance`)**: 생계의료비를 지원받는 인구수  
- **전세 지원 사업 여부 (`rental_support_area`)**: 전세 지원 사업 시행 여부 (True/False)  
- **계약 건수 (`contracts`)**: 해당 지역의 총 계약 건수  
- **민원 건수 (`complaints`)**: 해당 지역의 민원 건수  

데이터 원본: https://www.kaggle.com/datasets/anthonypino/melbourne-housing-market

In [ ]:
# 1. 패키지 불러오기 및 데이터 로드
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
data = pd.read_excel('3조_데이터.xlsx')

# 데이터 구조 보기
print("데이터 구조:")
print(data.info())
print("\n데이터 샘플:")
print(data.head())

In [7]:
# 2. 데이터 분리
# 예측에 필요하지 않은 변수 제거
X = data.drop(columns=['city', 'year', 'complaints', 'contracts'])  # 독립 변수
y = data['contracts']  # 종속 변수

# 수치형 변수만 선택 (Random Forest는 숫자 데이터를 요구)
X = pd.get_dummies(X, columns=['rental_support_area'], drop_first=True)

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
# 3. Random Forest Regressor 모델링
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

In [ ]:
# 4. 결과 평가
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-Squared (R²): {r2:.2f}")

In [ ]:
# 5. Feature Importance 시각화
importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=importances, y=features)
plt.title('Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

In [14]:
# 6. Complaints Ratio 계산
complaints_ratio = data.groupby('city').apply(
    lambda x: x['complaints'].sum() / x['contracts'].sum()
).reset_index()
complaints_ratio.columns = ['city', 'complaints_ratio']

# Complaints Ratio를 데이터에 추가
data = data.merge(complaints_ratio, on='city', how='left')

In [ ]:
# 7. Complaints 예측
# X_test의 인덱스를 y_pred와 매칭하기 위해 y_test의 인덱스를 사용
y_pred_complaints = y_pred * data.loc[y_test.index, 'complaints_ratio'].values

# Complaints 예측 결과 반올림
y_pred_complaints = np.round(y_pred_complaints).astype(int)

# Complaints 예측 결과 출력
predicted_complaints = pd.DataFrame({
    'city': data.loc[y_test.index, 'city'].values,
    'predicted_contracts': np.round(y_pred).astype(int),
    'predicted_complaints': y_pred_complaints
})
print(predicted_complaints.head())